In [ ]:
############### EDIT CONFIGURATION BELOW ###############

# BOUND_TL: TOP LEFT CAPTURE BOUNDARY
# BOUND_BR: BOTTOM RIGHT CAPTURE BOUNDARY
BOUND_TL = [40.19, -123.30]
BOUND_BR = [36.18, -119.26]

##################### IGNORE BELOW #####################
# BOUND_TL = [38.15, -122.98]     # SFO
# BOUND_BR = [36.99, -121.50]     # SFO
# BOUND_TL = [38.23, -123.25]     # NCT B
# BOUND_BR = [36.26, -119.08]     # NCT B
# BOUND_TL = [40.19, -123.30]     # NCT
# BOUND_BR = [36.18, -119.26]     # NCT
# BOUND_TL = [37.36, -120.88]     # FAT 
# BOUND_BR = [35.58, -117.97]     # FAT
# BOUND_TL = [42.06, -128.57]     # ZOA 
# BOUND_BR = [34.78, -115.52]     # ZOA
# BOUND_TL = [29.15, -88.02]      # ZMA 
# BOUND_BR = [23.46, -75.26]      # ZMA
# BOUND_TL = [51.06, -133.30]     # USA 
# BOUND_BR = [21.44, -59.50]      # USA
# BOUND_TL = [39.25, -77.40]      # DCA
# BOUND_BR = [38.23, -76.65]      # DCA

# MAP_AIRPORTS = ['KSFO', 'KOAK', 'KSJC', 'KMRY', 'KSMF', 'KRNO']
# MAP_AIRPORTS = ['KMIA', 'KFLL', 'KTPA', 'KPBI', 'KRSW', 'KEYW']
# MAP_AIRPORTS = ['KPHX', 'KLAX', 'KSAN', 'KSFO', 'KDEN', 'KMCO', \
#                'KMIA', 'KTPA', 'KATL', 'PHNL', 'KORD', 'KCVG', \
#                'KMRY', 'KBWI', 'KBOS', 'KDTW', 'KMSP', 'KMCI', \
#                'KSTL', 'KLAS', 'KEWR', 'KJFK', 'KLGA', 'KCLT', \
#                'KCLE', 'KPHL', 'KPIT', 'KMEM', 'KDAL', 'KDFW', \
#                'KHOU', 'KIAH', 'KSLC', 'KDCA', 'KIAD', 'KSEA']
##################### IGNORE ABOVE #####################

# MIN_ALT: MINUMUM ALTITUDE CAPTURED
# MAX_ALT: MAXIMUM ALTITUDE CAPTURED
MIN_ALT = 0
MAX_ALT = 24000

# DEP_FILTER: INCLUDED DEPARTURE AIRPORTS
# ARR_FILTER: INCLUDED ARRIVAL AIRPORTS
# INVERT_FILTER: CHANGE DEP/ARR FILTERS TO EXCLUDE
DEP_FILTER = ['KSFO', 'KOAK', 'KSQL', 'KSJC']
ARR_FILTER = ['KSFO', 'KOAK', 'KSQL', 'KSJC']
INVERT_FILTER = False

# MAP_AIRPORTS: AIRPORTS DISPLAYED ON UI MAP
# MAP_FIXES: FIXES DISPLAYED ON UI MAP
# MAP_NAVAIDS: NAVAIDS DISPLAYED ON UI MAP
MAP_AIRPORTS = ['KSFO', 'KOAK', 'KSJC']
MAP_FIXES = ['EDDYY', 'ARCHI', 'DUMBA']
MAP_NAVAIDS = ['OSI']

# D_TIME: DURATION BETWEEN CAPTURES (MIN)
# D_CAPTURE: DURATION OF TOTAL CAPTURES (MIN)
D_TIME = 1 * 60
D_CAPTURE = 1

############### EDIT CONFIGURATION ABOVE ###############

In [ ]:
########################################################
#################### VERSION 2.1.4 #####################
########################################################

################## DO NOT EDIT BELOW ###################

import time, os, pathlib, csv, json, subprocess, sys, re

import importlib.util as il
if None in [il.find_spec('FlightRadar24'), il.find_spec('numpy'), \
            il.find_spec('requests'), il.find_spec('pandas'), \
            il.find_spec('matplotlib'), il.find_spec('IPython')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'FlightRadarAPI']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'numpy']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'requests']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'pandas']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'matplotlib']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'IPython']);
os.system('cls')

from FlightRadar24 import FlightRadar24API
import numpy as np
import requests, urllib
import pandas as pd
import matplotlib as mpl, matplotlib.pyplot as plt
from IPython import display

fr_api = FlightRadar24API()

ft_config = fr_api.get_flight_tracker_config()
ft_config.gnd = '0'
ft_config.vehicles = '0'
fr_api.set_flight_tracker_config(ft_config)

bounds = f'{BOUND_TL[0]},{BOUND_BR[0]},{BOUND_TL[1]},{BOUND_BR[1]}'

def between(text, start, end):
    try: 
        return text.split(start)[1].split(end)[0]
    except Exception:
        return ''

def get_flight_details(id):
    base_url = 'https://data-live.flightradar24.com/clickhandler/?flight={}'
    data_url = base_url.format(id)
    req = urllib.request.Request(data_url, headers={'User-Agent': 'Magic Browser'})
    
    n_retry = 0
    con = None
    while con is None:
        try:
            con = urllib.request.urlopen(req, timeout=(50 + 10 * n_retry))
        except urllib.request.HTTPError as e:
            if n_retry > 5:
                return None
            n_retry += 1
    return json.loads(con.read().decode('utf8'))

def get_flights(prev_flights=[]):
    flights_unfilt = fr_api.get_flights(bounds=bounds)
    t0 = int(time.time())
    flights = []

    i = 0
    for i in range(0, len(flights_unfilt)):
        try:
            if flights_unfilt[i].callsign in prev_flights:
                continue
            details = get_flight_details(flights_unfilt[i].id)
            if details == None:
                time.sleep(3)
                i -= 1
                continue
        except Exception as e:
            if 'too many calls' not in str(e):
                print(e)
            time.sleep(3)
            i -= 1
            continue
        
        acft = {}
        acft['call'] = details['identification']['callsign']
        if 'model' in details['aircraft']:
            acft['type'] = details['aircraft']['model']['code']
        else:
            acft['type'] = 'ZZZZ'
        if 'registration' in details['aircraft']:
            acft['reg'] = details['aircraft']['registration']
        else:
            acft['reg'] = 'Blocked'

        if acft['call'] == None:
            acft['call'] = acft['reg']
        
        if acft['call'] in prev_flights:
            continue

        acft['dep'], acft['dep_gate'] = '','UNKN'
        if details['airport']['origin'] != None:
            acft['dep'] = details['airport']['origin']['code']['icao']
            acft['dep_gate'] = details['airport']['origin']['info']['gate']

        acft['arr'], acft['arr_gate'] = '','UNKN'
        if details['airport']['destination'] != None:
            acft['arr'] = details['airport']['destination']['code']['icao']
            acft['arr_gate'] = \
                details['airport']['destination']['info']['gate']

        if not INVERT_FILTER:
            if len(DEP_FILTER) == 0:
                if len(ARR_FILTER) != 0:
                    if acft['arr'] not in ARR_FILTER:
                        continue
            else:
                if len(ARR_FILTER) == 0:
                    if acft['dep'] not in DEP_FILTER:
                        continue
                else:
                    if acft['dep'] not in DEP_FILTER and \
                        acft['arr'] not in ARR_FILTER:
                        continue
        else:
            if acft['dep'] in DEP_FILTER or acft['arr'] in ARR_FILTER:
                continue
            
        acft['time_dep'] = details['time']['real']['departure']
        acft['time_arr'] = details['time']['estimated']['arrival']

        if acft['call'] == 'Blocked':
            continue
        elif len(details['trail']) < 4:
            continue
        elif details['trail'][0]['alt'] > MAX_ALT:
            continue
        elif details['trail'][0]['alt'] < MIN_ALT:
            continue

        if acft['reg'] == None:
            acft['reg'] = ''
        if acft['type'] == None:
            acft['type'] = 'ZZZZ'
        if acft['dep_gate'] == None:
            acft['dep_gate'] = 'UNKN'
        if acft['arr_gate'] == None:
            acft['arr_gate'] = 'UNKN'
        if acft['time_dep'] == None:
            acft['time_dep'] = -1
        if acft['time_arr'] == None:
            acft['time_arr'] = -1

        min_0, min_6 = 100000, 100000
        idx_0, idx_6 = 0, 0
        for j in range(0, len(details['trail'])):
            ts = details['trail'][j]['ts']
            dt = t0 - ts
            if abs(dt) < min_0:
                min_0 = abs(dt)
                idx_0 = j
            if abs(dt - 360) < min_6:
                min_6 = abs(dt - 360)
                idx_6 = j
        acft['trails'] = details['trail'][idx_0:idx_6]
        
        if details['trail'][idx_0]['ts'] - t0 < -5 and len(prev_flights) == 0:
            details['trail'][idx_0]['ts'] = t0 - 5
            
        pos = {}
        if len(prev_flights) > 0 and acft['trails'][-1]['spd'] < 30:
            pos_idx = len(acft['trails']) - 1
            for j in range(pos_idx, -1, -1):
                if acft['trails'][j]['spd'] > 30:
                    pos_idx = j
                    break
            pos = acft['trails'][pos_idx]
            pos['vs'] = 0
        else:
            pos = details['trail'][idx_0]
            if idx_0 + 3 >= len(details['trail']):
                idx_0 -= 3
            d_ts = details['trail'][idx_0]['ts'] - details['trail'][idx_0 + 3]['ts']
            d_alt = details['trail'][idx_0]['alt'] - details['trail'][idx_0 + 3]['alt']
            pos['vs'] = int(round(d_alt / (d_ts / 60), -2))
        acft['pos'] = pos
        
        acft['ias'] = round(pos['spd'] / (1 + (pos['alt'] * 0.00002)))
        acft['mach'] = round(pos['spd'] / ((1.4 * 287.053 * \
            ((-6.49e-3 * pos['alt'] * .3048 + 14.9855) + 273.15)) \
                ** .5 * 1.944), 2)

        url_base = 'https://www.flightaware.com/live/flight/'
        r = requests.get(url_base + acft['call'])
        fa_data = r.text
        if r'"route"' in fa_data:
            temp_text = fa_data[fa_data.rindex(r'"route"') - 3000:
                               fa_data.rindex(r'"route"') + 1500]
            flight_plan = between(temp_text, 
                                  r'"flightPlan":', r'"fuelBurn"')
            acft['rte'] = between(flight_plan, r'"route":"', r'",') \
                .replace(',', '')
            acft['fp_spd'] = '0' + between(flight_plan, 
                                r'"speed":', r',')
            acft['fp_spd'] = int(acft['fp_spd'].replace('null', '0'))
            acft['fp_alt'] = '0' + between(flight_plan, 
                                r'"altitude":', r',')
            acft['fp_alt'] = int(acft['fp_alt'].replace('null', '0')) * 100
        else:
            acft['rte'], acft['fp_spd'], acft['fp_alt'] = '', 0, 0
            
        acft['cmd'] = ''

        flights.append(acft)
        print(f'({i + 1}/{len(flights_unfilt)})\tAdding {acft["call"]}.')
                        
    return flights

def sort_flights(flights):
    flights = sorted(flights, key=lambda d: d['pos']['ts'])
    for flight in flights:
        flight['s'] = flight['pos']['ts'] - flights[0]['pos']['ts']
    return flights

def get_airports(airports):
    data_url = 'https://www.flightradar24.com/_json/airports.php'
    req = urllib.request.Request(data_url, headers={'User-Agent': 'Magic Browser'})
    
    n_retry = 0
    con = None
    while con is None:
        try:
            con = urllib.request.urlopen(req, timeout=(50 + 10 * n_retry))
        except urllib.request.HTTPError as e:
            if n_retry > 5:
                return []
            n_retry += 1
    data = json.loads(con.read().decode('utf8'))['rows']

    airports_out = []
    for apt in data:
        if apt.get('icao') in airports:
            airports_out.append(apt)
    return airports_out

def get_fixes(fixes, navaids=False):
    fixes_out = []
    base_url = 'http://www.myfsim.com/fix.php?FIX={}'
    if navaids:
        base_url = 'http://www.myfsim.com/navaid.php?NAVAID={}'
    for fix in fixes:
        data_url = base_url.format(fix)
        req = urllib.request.Request(data_url, headers={'User-Agent': 'Magic Browser'})
        
        n_retry = 0
        con = None
        while con is None:
            try:
                con = urllib.request.urlopen(req, timeout=(50 + 10 * n_retry))
            except urllib.request.HTTPError as e:
                if n_retry > 5:
                    continue
                n_retry += 1
        data = con.read().decode('utf8')
        if 'Coords' not in data:
            continue
        lat, lon = re.sub(r'[^0-9\.\-/]', '', data.split('Coords')[1]).split('/')
        fixes_out.append({'name': fix, 'lat': float(lat), 'lon': float(lon)})

    return fixes_out

def plot_aircraft(flight, ax, bright=0, text=True):
    A = [.1, .3, .5, .6, .7]
    if bright == -1:
        A = np.array(A) * .5
    elif bright == 1:
        A = [.5, 1, 1, 1, 1]
    y = flight['pos']['lat']
    x = flight['pos']['lng']
    tr_y = [d['lat'] for d in flight['trails']]
    tr_x = [d['lng'] for d in flight['trails']]
    call = flight['call']
    hdg = (360 - flight['pos']['hd']) % 360
    alt = str(int(round(flight['pos']['alt'], -2) / 100)).zfill(3)
    vs = '$=\\!$' 
    if flight['pos']['vs'] > 0:
        vs = '$\\uparrow\\!\\!$'
    elif flight['pos']['vs'] < 0:
        vs = '$\\downarrow\\!\\!$'
    spd = str(flight['pos']['spd']).zfill(3)
    dep, arr = flight['dep'], flight['arr']
    spawn = flight['s']

    # Plot aircraft trail
    l_tr = ax.plot(tr_x, tr_y, alpha=A[0])[0]

    # Plot aircraft arrow and label
    t = mpl.markers.MarkerStyle(marker='^')
    t._transform = t.get_transform().rotate_deg(hdg)
    ax.plot(x, y, c=l_tr.get_color(), marker=t, markersize=5, alpha=A[2])
    t = mpl.markers.MarkerStyle(marker='|')
    t._transform = t.get_transform().rotate_deg(hdg)
    ax.plot(x, y, c=l_tr.get_color(), marker=t, markersize=10, alpha=A[2])
    if not text:
        return
    ax.annotate(call, [x, y], xytext=[5, 0], 
                textcoords='offset points', fontsize=10, alpha=A[4])
    
    # Plot VSI, altitude, speed
    ax.annotate(vs + alt, [x, y], xytext=[1.5, -7.5], 
                textcoords='offset points', fontsize=8, alpha=A[1])
    ax.annotate(spd, [x, y], xytext=[30, -7.5], 
                textcoords='offset points', fontsize=8, alpha=A[1])

    # Plot departure/arrival airport
    ax.annotate(dep, [x, y], xytext=[5, -15], 
                textcoords='offset points', fontsize=8, alpha=A[1])
    ax.annotate(arr, [x, y], xytext=[30, -15], 
                textcoords='offset points', fontsize=8, alpha=A[1])

    # Plot spawn delay
    ax.annotate(f'({spawn})', [x, y], xytext=[5, 10], 
                textcoords='offset points', fontsize=5, alpha=A[3])

airports = get_airports(MAP_AIRPORTS)
fixes = get_fixes(MAP_FIXES)
navaids = get_fixes(MAP_NAVAIDS, True)

def plot_waypoints(ax):
    for apt in airports:
        y, x, iata = apt['lat'], apt['lon'], apt['iata']
        ax.plot(x, y, c='r', marker='x', markersize=5, zorder=999)
        ax.annotate(iata, [x, y], xytext=[5, 0],
                    textcoords='offset points', 
                    c='r', fontsize=10, alpha=.5, zorder=999)
    
    for fix in fixes:
        y, x, name = fix['lat'], fix['lon'], fix['name']
        ax.plot(x, y, c='k', marker='+', markersize=5, zorder=999)
        ax.annotate(name, [x, y], xytext=[5, 0],
                    textcoords='offset points', 
                    c='k', fontsize=8, alpha=.4, zorder=999)

    for fix in navaids:
        y, x, name = fix['lat'], fix['lon'], fix['name']
        ax.scatter(x, y, edgecolors='k', s=5, facecolors='none', zorder=999)
        ax.annotate(name, [x, y], xytext=[5, 0],
                    textcoords='offset points', 
                    c='k', fontsize=8, alpha=.4, zorder=999)

def plot_map(flights, bright=[], clear=True, dpi=600, 
             goal_s=-1, spread=[3, 6]):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    if clear:
        fig.canvas.draw()
        fig.canvas.flush_events()
        ax.cla()
        display.clear_output(wait=False)
    
    plt.rcParams['font.family'] = 'monospace'
    plt.rcParams['figure.dpi'] = dpi
    plt.ion()
    
    for flight in flights:
        if goal_s != -1:
            s = flight['s']
            if s < goal_s - spread[0] * 60 or s > goal_s + spread[1] * 60:
                continue
        if len(bright) == 0:
            plot_aircraft(flight, ax, bright=0)
        elif flight['call'] in bright:
            plot_aircraft(flight, ax, bright=1)
        else:
            plot_aircraft(flight, ax, bright=-1)
            
    plot_waypoints(ax)

    b = [float(b) for b in bounds.split(',')]
    b[0] += .1
    b[1] -= .1
    b[2] -= .1
    b[3] += .1
    if b[2] < ax.get_xlim()[0]:
        b[2] = ax.get_xlim()[0]
    if b[3] > ax.get_xlim()[1]:
        b[3] = ax.get_xlim()[1]
    if b[0] > ax.get_ylim()[1]:
        b[0] = ax.get_ylim()[1]
    if b[1] < ax.get_ylim()[0]:
        b[1] = ax.get_ylim()[0]
    ax.set_xlim(b[2], b[3])
    ax.set_ylim(b[1], b[0])
    
    plt.pause(.001)
    plt.show()

def capture_commands(flights):
    flights = sort_flights(flights)
    plot_map(flights, clear=False)
    
    info_text = 'Next: \'+\' (or blank)\tPrev: \'-\'' \
        + '\tExit: \'\\\'' \
        + '\nInput multiple commands separated by semicolons (\';\')' \
        + '\nSpecial Commands:  \'DEL\'     - delete aircraft' \
        + '\n\t\t   \'DEP/XXX\' - departure from XXX airport' \
        + '\n\t\t   \'*FIXXX,ABCDE,...\' - toggle fixes\n\n'
    input('Press \'Enter\' to continue.\n')

    pos = 0
    while True:
        if pos == -1:
            plot_map(flights)
            inp = input(info_text)
            pos += 1
            continue
        elif pos == len(flights):
            plot_map(flights)
            inp = input('Press \'Enter\' to exit.\n')
            plt.close('all')
            break
        elif pos < -1:
            pos == -1
        else:
            flight = flights[pos]
            plot_map(flights, bright=[flight['call']], 
                     dpi=150, goal_s=flight['s'])
            acft_info = f'[{pos + 1}/{len(flights)}] | ' + \
                f'{flight["call"]} | {flight["type"]} | ' + \
                f'{flight["dep"]} - {flight["arr"]} | '+ \
                f'{str(int(flight["fp_alt"] / 100)).zfill(3)} | ' + \
                f'{flight["rte"]}'
            inp = input(info_text + f'{acft_info}\n')
            if inp == '-':
                pos -= 1
                continue
            elif inp == '+' or len(inp) == 0:
                pos += 1
                continue
            elif inp == '\\':
                plot_map(flights)
                break
            elif inp.lower() == 'del':
                flights.pop(pos)
                continue
            elif inp[0] == '*':
                inp = inp.upper()
                inp = re.sub(r'[^A-Z\,]', '', inp)
                new_fixes = inp.split(',')
                global MAP_FIXES, fixes
                for fix in new_fixes:
                    if fix in MAP_FIXES:
                        MAP_FIXES.remove(fix)
                    else:
                        MAP_FIXES.append(fix)
                fixes = get_fixes(MAP_FIXES)
                continue
            else:
                flights[pos]['cmd'] = inp.upper()
                pos += 1
                continue
                
def dump_flights(flights):
    out_dir = os.path.join(str(pathlib.Path.home() / 'Downloads'), \
                'FAST' + os.sep + 'dumps')
    out_file = 'DUMP_' + time.strftime('%y%m%d-%H%M', \
                    time.gmtime(flights[0]['pos']['ts'])) + '.json'
    out_file = os.path.join(out_dir, out_file)

    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    with open(out_file, 'w') as f:
        json.dump(flights , f)
        
def read_flights(in_file):
    in_dir = os.path.join(str(pathlib.Path.home() / 'Downloads'), \
                'FAST' + os.sep + 'dumps')
    in_file = os.path.join(in_dir, in_file)
    
    with open(in_file, 'r') as f:
        data = json.load(f)
    
    return data

def write_data(flights):
    do_write = input('Press \'Enter\' to save data. ' + \
              'Input any text to exit without saving.\n')
    if len(do_write) != 0:
        return
    
    out_dir = os.path.join(str(pathlib.Path.home() / 'Downloads'), \
                'FAST' + os.sep + 'scenarios')
    out_file = 'FAST_' + time.strftime('%y%m%d-%H%M', \
                    time.gmtime(flights[0]['pos']['ts'])) + '.csv'
    out_file = os.path.join(out_dir, out_file)

    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    with open(out_file, 'w') as f: 
        header = 'callsign,delay,type,reg,dep,arr,dep_gate,arr_gate,' \
                    + 'lat,lon,alt,spd,ias,mach,hdg,rte,fp_spd,fp_alt,cmd\n'
        f.write(header)
        for a in flights:
            p = a['pos']
            data = [a['call'], a['s'], a['type'], a['reg'], a['dep'], \
                    a['arr'], a['dep_gate'], a['arr_gate'], p['lat'], \
                    p['lng'], p['alt'], p['spd'], a['ias'], a['mach'], \
                    p['hd'], a['rte'], a['fp_spd'], a['fp_alt'], a['cmd']]
            
            f.write(','.join(str(v) for v in data) + '\n')
            
################## DO NOT EDIT ABOVE ###################

In [ ]:
############ RUN CELL TO CAPTURE LIVE DATA #############

flights = []
n_cycles = int(D_CAPTURE * 60 / D_TIME)
for i in range(0, n_cycles):
    t0 = time.time()
    for new_flight in get_flights([flight['call'] for flight in flights]):
        flights.append(new_flight)
    delay = int(D_TIME - (time.time() - t0))
    if i < n_cycles - 1:
        for j in range(0, delay):
            print(f'[{i + 1}/{n_cycles}] Waiting {delay - j} ' + \
                  'seconds to capture more data.          ', end='\r')
            time.sleep(1)
        print('--------------------------------------------------')

In [ ]:
############# EDIT TO UPLOAD/DOWNLOAD DATA #############

# UNCOMMENT TO OUTPUT FLIGHT DATA TO 'Downloads/FAST/dumps'
# dump_flights(flights)

# UNCOMMENT AND EDIT FILE NAME TO LOAD FLIGHT DATA 
# FROM 'Downloads/FAST/dumps/(DUMP_######-####.json)'
# flights = read_flights('DUMP_231129-0537.json')

############### EDIT CELL ONLY IF NEEDED ###############

In [ ]:
########### RUN CELL TO MODIFY/SAVE SCENARIO ###########
capture_commands(flights)
write_data(flights)